In [ ]:
import pandas as pd
import numpy as np
import joblib
import logging
import os
from sklearn.preprocessing import MinMaxScaler

# --- Configuration ---
BASE_DATA_DIR = os.getenv('DATA_DIR', '/path/to/project/data')  # Example: DATA_DIR="./Data"

# Indian Air Quality (Training) - Using city_hour.csv
INDIA_AQ_DIR = os.path.join(BASE_DATA_DIR, 'India') # Directory containing city_hour.csv
INDIA_TRAIN_CITY = 'Delhi' # Confirmed this worked in your log for loading

# Beijing Air Quality (Testing)
BEIJING_AQ_STATIONS_DIR = os.path.join(BASE_DATA_DIR, 'China/station_data')
BEIJING_TEST_STATION_FILE = 'PRSA_Data_Aotizhongxin_20130301-20170228.csv' # Example station file

# Output paths
PREPROCESSED_TRAIN_PATH = os.path.join(BASE_DATA_DIR, 'train_pollutants_preprocessed.csv')
PREPROCESSED_TEST_PATH = os.path.join(BASE_DATA_DIR, 'test_pollutants_preprocessed.csv')
SCALER_FILE_PATH = os.path.join(BASE_DATA_DIR, 'pollutants_scaler.pkl')

SELECTED_POLLUTANTS_INDIA = ['PM2.5', 'NO2', 'CO']
SELECTED_POLLUTANTS_BEIJING = ['PM2.5', 'NO2', 'CO'] # Column names in Beijing data
RENAMED_FEATURES = ['pm25', 'no2', 'co']

ROLLING_WINDOW_SIZE = 5
IQR_MULTIPLIER = 1.5

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_preprocess_india_aq_pollutants(data_dir, city_to_filter, pollutant_cols_original_names):
    logging.info(f"Attempting to load Indian AQ data for city: {city_to_filter} from dir: {data_dir} using city_hour.csv")
    city_hour_file = os.path.join(data_dir, 'city_hour.csv')

    if not os.path.exists(city_hour_file):
        logging.error(f"File not found: {city_hour_file}.")
        return pd.DataFrame()
        
    try:
        preview_df = pd.read_csv(city_hour_file, nrows=5)
        available_cols = preview_df.columns.tolist()
        logging.info(f"Available columns in city_hour.csv: {available_cols}")

        actual_use_cols = ['City', 'Datetime']
        final_rename_map = {'Datetime': 'timestamp'}
        
        valid_pollutants_found = True
        for i, original_name in enumerate(pollutant_cols_original_names):
            if original_name in available_cols:
                actual_use_cols.append(original_name)
                final_rename_map[original_name] = RENAMED_FEATURES[i]
            else:
                logging.error(f"Expected pollutant column '{original_name}' not found in city_hour.csv.")
                valid_pollutants_found = False
        
        if not valid_pollutants_found or 'City' not in available_cols or 'Datetime' not in available_cols:
            logging.error("Essential columns for processing Indian AQ data are missing. Cannot proceed.")
            return pd.DataFrame()

        logging.info(f"Will use columns: {actual_use_cols} and rename map: {final_rename_map}")
        
        chunk_iter = pd.read_csv(city_hour_file, usecols=list(set(actual_use_cols)), chunksize=100000, low_memory=False)
        df_list = []
        for chunk in chunk_iter:
            df_list.append(chunk[chunk['City'] == city_to_filter])
        
        if not df_list:
             logging.warning(f"No data found for City = '{city_to_filter}' after chunk processing.")
             return pd.DataFrame()
        df_city = pd.concat(df_list)

        if df_city.empty:
            logging.warning(f"No data found for City = '{city_to_filter}'.")
            return pd.DataFrame()

        logging.info(f"Loaded {len(df_city)} records for City = {city_to_filter}.")
        df = df_city.rename(columns=final_rename_map)
        
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        df = df.dropna(subset=['timestamp'])

        for col in RENAMED_FEATURES:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.set_index('timestamp').sort_index()
        
        for col in RENAMED_FEATURES:
            if col in df.columns:
                df[col] = df[col].interpolate(method='time').ffill().bfill()
        
        df = df.dropna(subset=RENAMED_FEATURES)
        
        if df.empty:
            logging.warning(f"Data for {city_to_filter} became empty after cleaning pollutants.")
            return pd.DataFrame()
            
        logging.info(f"Preprocessed Indian AQ data for {city_to_filter}. Shape: {df.shape}")
        return df.reset_index()

    except Exception as e:
        logging.error(f"Error loading/preprocessing Indian AQ data: {e}", exc_info=True)
        return pd.DataFrame()

def load_and_preprocess_beijing_aq_pollutants(stations_data_dir, station_filename, pollutant_cols_original_names):
    # 'stations_data_dir' is now expected to be the direct path to the folder containing the station CSVs
    logging.info(f"Attempting to load Beijing AQ data from file: {station_filename} in dir: {stations_data_dir}")
    file_path = os.path.join(stations_data_dir, station_filename) # This construction should now be correct

    if not os.path.exists(file_path):
        logging.error(f"File not found: {file_path}. Please check the path and filename for Beijing AQ data.")
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']], errors='coerce')
        df = df.dropna(subset=['timestamp'])
        
        preview_cols = df.columns.tolist()
        logging.info(f"Available columns in Beijing station file {station_filename}: {preview_cols}")

        final_rename_map = {}
        actual_use_cols_beijing = ['timestamp'] 

        valid_pollutants_found_beijing = True
        for i, original_name in enumerate(pollutant_cols_original_names): 
            if original_name in preview_cols:
                actual_use_cols_beijing.append(original_name)
                final_rename_map[original_name] = RENAMED_FEATURES[i]
            else:
                logging.error(f"Expected pollutant column '{original_name}' not found in Beijing station file {station_filename}.")
                valid_pollutants_found_beijing = False

        if not valid_pollutants_found_beijing:
            logging.error("Essential pollutant columns missing in Beijing data. Cannot proceed.")
            return pd.DataFrame()
        
        df = df[actual_use_cols_beijing].rename(columns=final_rename_map)
        
        for col in RENAMED_FEATURES: 
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.set_index('timestamp').sort_index()
        
        for col in RENAMED_FEATURES:
            if col in df.columns:
                df[col] = df[col].interpolate(method='time').ffill().bfill()
        
        df = df.dropna(subset=RENAMED_FEATURES)
        
        if df.empty:
            logging.warning(f"Beijing data from {station_filename} became empty after cleaning pollutants.")
            return pd.DataFrame()

        logging.info(f"Preprocessed Beijing AQ data from {station_filename}. Shape: {df.shape}")
        return df.reset_index()

    except Exception as e:
        logging.error(f"Error loading/preprocessing Beijing AQ data from {station_filename}: {e}", exc_info=True)
        return pd.DataFrame()

def engineer_features_pollutants(df, features_to_process, rolling_window=ROLLING_WINDOW_SIZE):
    if df.empty or 'timestamp' not in df.columns:
        logging.warning("DataFrame is empty or missing 'timestamp' in engineer_features_pollutants.")
        return pd.DataFrame()
    df_processed = df.copy()
    
    df_processed['timestamp'] = pd.to_datetime(df_processed['timestamp'], errors='coerce')
    df_processed = df_processed.dropna(subset=['timestamp'])
    if df_processed.empty: return pd.DataFrame()

    df_processed['hour'] = df_processed['timestamp'].dt.hour
    df_processed['day_of_week'] = df_processed['timestamp'].dt.dayofweek
    df_processed['month'] = df_processed['timestamp'].dt.month # Added month
    
    df_processed = df_processed.sort_values(by='timestamp')
    
    for feature in features_to_process: 
        if feature in df_processed.columns:
            df_processed[feature] = pd.to_numeric(df_processed[feature], errors='coerce')
            # It's better to handle NaNs before this point or ensure they don't cause issues in rolling/lag
            # df_processed = df_processed.dropna(subset=[feature]) # This might drop too much if previous steps didn't fill
            # if df_processed.empty : break 

            df_processed[f'{feature}_rolling_mean_{rolling_window}'] = df_processed[feature].rolling(window=rolling_window, min_periods=1).mean()
            df_processed[f'{feature}_lag1'] = df_processed[feature].shift(1)
            if not df_processed.empty:
                 df_processed[f'{feature}_lag1'].fillna(df_processed[feature].iloc[0] if not df_processed.empty else np.nan, inplace=True)
        else:
            logging.warning(f"Feature '{feature}' not found for engineering in engineer_features_pollutants.")
    
    # Drop rows that have NaNs in the newly created lag/rolling features if any (should be minimal if filled)
    df_processed = df_processed.dropna() 
    return df_processed.reset_index(drop=True)

def detect_outliers_iqr(df, column, multiplier=IQR_MULTIPLIER):
    if df.empty or column not in df.columns or df[column].isnull().all():
        return pd.Series([False] * len(df.index), index=df.index if not df.empty else None) # Ensure index is passed
    numeric_col = pd.to_numeric(df[column], errors='coerce')
    if numeric_col.isnull().all():
        return pd.Series([False] * len(df.index), index=df.index)

    Q1 = numeric_col.quantile(0.25)
    Q3 = numeric_col.quantile(0.75)
    IQR = Q3 - Q1
    if IQR == 0 or pd.isna(IQR): return pd.Series([False] * len(df.index), index=df.index)
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return (numeric_col < lower_bound) | (numeric_col > upper_bound)

if __name__ == "__main__":
    if not os.path.exists(BASE_DATA_DIR):
        os.makedirs(BASE_DATA_DIR, exist_ok=True)
        logging.info(f"Base data directory created/confirmed: {BASE_DATA_DIR}")
    
    # Check existence of the specific directories where data files are expected
    if not os.path.exists(INDIA_AQ_DIR):
        logging.error(f"Indian AQ data directory not found: {INDIA_AQ_DIR} (expects city_hour.csv here)")
    if not os.path.exists(BEIJING_AQ_STATIONS_DIR):
        logging.error(f"Beijing AQ stations directory not found: {BEIJING_AQ_STATIONS_DIR} (expects station CSV files here)")

    logging.warning(f"Using INDIA_TRAIN_CITY: '{INDIA_TRAIN_CITY}'. Ensure this city has data for pollutants {SELECTED_POLLUTANTS_INDIA} in '{os.path.join(INDIA_AQ_DIR, 'city_hour.csv')}'")

    train_df_raw = load_and_preprocess_india_aq_pollutants(INDIA_AQ_DIR, INDIA_TRAIN_CITY, SELECTED_POLLUTANTS_INDIA)
    # The BEIJING_AQ_STATIONS_DIR should correctly point to the folder containing the CSVs
    test_df_raw = load_and_preprocess_beijing_aq_pollutants(BEIJING_AQ_STATIONS_DIR, BEIJING_TEST_STATION_FILE, SELECTED_POLLUTANTS_BEIJING)

    if train_df_raw.empty:
        logging.error("Training data is empty after loading. Halting preprocessing.")
    if test_df_raw.empty:
        logging.error("Testing data is empty after loading. Halting preprocessing.")
        
    if not train_df_raw.empty and not test_df_raw.empty:
        logging.info("Engineering features for training data...")
        train_df_featured = engineer_features_pollutants(train_df_raw.copy(), RENAMED_FEATURES)
        logging.info("Engineering features for testing data...")
        test_df_featured = engineer_features_pollutants(test_df_raw.copy(), RENAMED_FEATURES)

        if train_df_featured.empty and test_df_featured.empty: # Check if BOTH are empty
             logging.error("Both training and testing data are empty after feature engineering.")
        elif train_df_featured.empty:
            logging.error("Training data became empty after feature engineering.")
        elif test_df_featured.empty:
            logging.error("Testing data became empty after feature engineering.")
        else: # Proceed if at least training data is not empty
            cols_to_scale_base = RENAMED_FEATURES
            cols_to_scale_engineered = []
            for feature_base_name in RENAMED_FEATURES:
                cols_to_scale_engineered.append(f'{feature_base_name}_rolling_mean_{ROLLING_WINDOW_SIZE}')
                cols_to_scale_engineered.append(f'{feature_base_name}_lag1')
            
            cols_to_scale = cols_to_scale_base + cols_to_scale_engineered
            final_cols_to_scale_train = [col for col in cols_to_scale if col in train_df_featured.columns and pd.api.types.is_numeric_dtype(train_df_featured[col])]
            
            scaler = MinMaxScaler()
            if final_cols_to_scale_train:
                logging.info(f"Fitting scaler on training data columns: {final_cols_to_scale_train}")
                train_df_featured[final_cols_to_scale_train] = scaler.fit_transform(train_df_featured[final_cols_to_scale_train])
                joblib.dump(scaler, SCALER_FILE_PATH)
                logging.info(f"Scaler saved to {SCALER_FILE_PATH}")

                if not test_df_featured.empty: # Only scale test if it's not empty
                    final_cols_to_scale_test = [col for col in final_cols_to_scale_train if col in test_df_featured.columns and pd.api.types.is_numeric_dtype(test_df_featured[col])]
                    if final_cols_to_scale_test:
                        missing_in_test = set(final_cols_to_scale_test) - set(test_df_featured.columns)
                        if missing_in_test:
                            logging.warning(f"Columns {missing_in_test} needed for scaling are missing in test data.")
                            final_cols_to_scale_test = [col for col in final_cols_to_scale_test if col not in missing_in_test]
                        
                        if final_cols_to_scale_test and not test_df_featured[final_cols_to_scale_test].isnull().values.any():
                            logging.info(f"Transforming testing data columns: {final_cols_to_scale_test}")
                            test_df_featured[final_cols_to_scale_test] = scaler.transform(test_df_featured[final_cols_to_scale_test])
                        elif not final_cols_to_scale_test:
                             logging.warning("No valid columns left to scale in test data after checks.")
                        else:
                            logging.warning(f"Testing data columns for scaling contain NaNs. Investigate. Skipping scaling for test data.")
                    else:
                        logging.warning("No common numeric columns to scale found in testing data (or test_df_featured empty).")
            else:
                logging.error("No numeric columns found for scaling in training data. Scaler not fitted.")

            train_df_featured['is_outlier'] = False 
            for i, feature in enumerate(RENAMED_FEATURES): 
                 if feature in train_df_featured.columns:
                    outlier_col_name = f'is_outlier_{feature}'
                    train_df_featured[outlier_col_name] = detect_outliers_iqr(train_df_featured, feature)
                    train_df_featured['is_outlier'] = train_df_featured['is_outlier'] | train_df_featured[outlier_col_name]
            logging.info(f"Outlier labels generated for training data. Total anomalies found: {train_df_featured['is_outlier'].sum() if 'is_outlier' in train_df_featured else 'N/A'}")

            train_df_featured.to_csv(PREPROCESSED_TRAIN_PATH, index=False)
            logging.info(f"Preprocessed training data saved to {PREPROCESSED_TRAIN_PATH}")
            if not test_df_featured.empty : 
                test_df_featured.to_csv(PREPROCESSED_TEST_PATH, index=False)
                logging.info(f"Preprocessed testing data saved to {PREPROCESSED_TEST_PATH}")
            else:
                logging.warning("Test data featured is empty, not saving.")

            logging.info("--- Preprocessing Script Finished ---")
            if not train_df_featured.empty:
                print("\n--- Preprocessed Training Data (Head) ---")
                print(train_df_featured.head())
            if not test_df_featured.empty:
                print("\n--- Preprocessed Testing Data (Head) ---")
                print(test_df_featured.head())
    else:
        logging.info("Preprocessing halted due to empty raw data for training or testing.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
# import joblib # Not strictly needed here unless re-loading scaler for other purposes
import logging
import os

# --- Configuration ---
BASE_DATA_DIR = os.getenv('DATA_DIR', '/path/to/project/data') # Where preprocessed files are stored
PREPROCESSED_TRAIN_PATH = os.path.join(BASE_DATA_DIR, 'train_pollutants_preprocessed.csv')
PREPROCESSED_TEST_PATH = os.path.join(BASE_DATA_DIR, 'test_pollutants_preprocessed.csv')
# SCALER_FILE_PATH = os.path.join(BASE_DATA_DIR, 'pollutants_scaler.pkl') # Path to saved scaler

EDA_OUTPUT_DIR = './pollutants_eda_plots/' # Directory to save plots

# Features to focus on for EDA (the renamed ones)
PRIMARY_POLLUTANT_FEATURES = ['pm25', 'no2', 'co']

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- EDA Plotting Functions ---
def plot_time_series_trends(df, df_name, features_to_plot, output_dir):
    if df.empty or not all(col in df.columns for col in features_to_plot) or 'timestamp' not in df.columns:
        logging.warning(f"DataFrame {df_name} is empty or missing required columns for time-series plot. Features expected: {features_to_plot}, Timestamp. Available: {df.columns.tolist()}")
        return

    try:
        df_plot = df.set_index('timestamp').copy()
        # Ensure features are numeric for plotting
        for feature in features_to_plot:
            df_plot[feature] = pd.to_numeric(df_plot[feature], errors='coerce')
        df_plot = df_plot.dropna(subset=features_to_plot) # Drop rows if conversion made NaNs
        if df_plot.empty :
            logging.warning(f"DataFrame {df_name} became empty after ensuring numeric types for plotting time series.")
            return
    except KeyError:
        logging.error(f"'timestamp' column not found in {df_name} for setting as index.")
        return
    except Exception as e:
        logging.error(f"Error preparing data for time-series plot for {df_name}: {e}")
        return


    plt.figure(figsize=(15, len(features_to_plot) * 3)) # Adjust height
    for i, feature in enumerate(features_to_plot):
        plt.subplot(len(features_to_plot), 1, i + 1)
        plt.plot(df_plot.index, df_plot[feature], label=feature, alpha=0.8)
        plt.title(f'{feature} Trend ({df_name}) - Scaled Values')
        plt.xlabel('Time')
        plt.ylabel(f'Scaled {feature}')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.suptitle(f'Time-Series Trends ({df_name})', fontsize=16, y=1.02)
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.savefig(os.path.join(output_dir, f'{df_name}_time_series_trends.png'))
    plt.close()
    logging.info(f"Time-series trend plot saved for {df_name}.")

def plot_correlation_matrix(df, df_name, features, output_dir):
    if df.empty or not features:
        logging.warning(f"DataFrame {df_name} is empty or no features provided for correlation matrix.")
        return
    
    # Ensure features are present and numeric
    valid_features = [f for f in features if f in df.columns]
    if not valid_features:
        logging.warning(f"None of the provided features for correlation exist in DataFrame {df_name}.")
        return

    numeric_df_for_corr = df[valid_features].apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')
    if numeric_df_for_corr.empty or numeric_df_for_corr.shape[1] < 2: # Need at least 2 columns for corr
        logging.warning(f"Not enough numeric features left for correlation matrix in {df_name} (need at least 2).")
        return

    correlation_matrix = numeric_df_for_corr.corr()
    plt.figure(figsize=(max(8, len(valid_features)*0.8), max(6, len(valid_features)*0.6))) # Dynamic size
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f", annot_kws={"size": 8})
    plt.title(f'Feature Correlation Matrix ({df_name})')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{df_name}_correlation_matrix.png'))
    plt.close()
    logging.info(f"Correlation matrix saved for {df_name}.")

def plot_distribution_analysis(df, df_name, features, output_dir):
    if df.empty or not features:
        logging.warning(f"DataFrame {df_name} is empty or no features provided for distribution plot.")
        return
    
    valid_features = [f for f in features if f in df.columns]
    if not valid_features:
        logging.warning(f"None of the provided features for distribution exist in DataFrame {df_name}.")
        return

    num_plots = len(valid_features)
    # Adjust subplot layout for many features
    cols_subplot = 2 if num_plots > 1 else 1
    rows_subplot = (num_plots + cols_subplot - 1) // cols_subplot

    plt.figure(figsize=(12, 5 * rows_subplot))
    for i, feature in enumerate(valid_features):
        plt.subplot(rows_subplot, cols_subplot, i + 1)
        try:
            feature_data = pd.to_numeric(df[feature], errors='coerce').dropna()
            if feature_data.empty:
                logging.warning(f"Feature '{feature}' in {df_name} is all NaN or non-numeric for distribution plot.")
                plt.text(0.5, 0.5, 'No numeric data for this feature', ha='center', va='center')
                plt.title(f'Distribution of {feature} (No Data)')
                continue
            sns.histplot(feature_data, kde=True, label=feature, alpha=0.7, bins=50)
            plt.title(f'Distribution of {feature}')
            plt.xlabel('Value')
            plt.ylabel('Frequency')
            plt.legend()
        except Exception as e:
            logging.error(f"Error plotting distribution for {feature} in {df_name}: {e}")
            plt.text(0.5, 0.5, f'Error plotting {feature}', ha='center', va='center')
            plt.title(f'Distribution of {feature} (Error)')


    plt.suptitle(f'Distribution of Features ({df_name}) - Scaled Values', fontsize=16, y=1.02 if num_plots > 1 else 1.05)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(os.path.join(output_dir, f'{df_name}_distribution_analysis.png'))
    plt.close()
    logging.info(f"Distribution analysis plot saved for {df_name}.")

def plot_box_plots(df, df_name, features, output_dir):
    if df.empty or not features:
        logging.warning(f"DataFrame {df_name} is empty or no features for box plots.")
        return

    valid_features = [f for f in features if f in df.columns]
    if not valid_features:
        logging.warning(f"None of the provided features for boxplot exist in DataFrame {df_name}.")
        return
        
    try:
        data_for_boxplot = df[valid_features].apply(pd.to_numeric, errors='coerce')
        # Drop columns that are all NaN after numeric conversion
        data_for_boxplot = data_for_boxplot.dropna(axis=1, how='all')
        if data_for_boxplot.empty:
            logging.warning(f"No numeric data left for box plots in {df_name}.")
            return

        plt.figure(figsize=(max(8, len(data_for_boxplot.columns) * 1.5), 6))
        sns.boxplot(data=data_for_boxplot)
        plt.title(f'Box Plots for Features ({df_name}) - Scaled Values')
        plt.ylabel('Scaled Values')
        plt.xticks(rotation=30, ha='right')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{df_name}_box_plots.png'))
        plt.close()
        logging.info(f"Box plots saved for {df_name}.")
    except Exception as e:
        logging.error(f"Error generating box plot for {df_name}: {e}")


def plot_feature_importance(X_train_df, y_train_series, df_name, output_dir):
    if X_train_df.empty or y_train_series.empty:
        logging.warning(f"Training data for feature importance ({df_name}) is empty.")
        return
    if len(X_train_df.columns) == 0:
        logging.warning(f"No features in X_train_df for {df_name}.")
        return
    try:
        model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced_subsample') # Added subsample
        model.fit(X_train_df, y_train_series.astype(int))
        importances = model.feature_importances_
        features = X_train_df.columns
        
        sorted_indices = np.argsort(importances)[::-1]

        plt.figure(figsize=(10, max(6, len(features) * 0.45))) # Dynamic height
        sns.barplot(x=importances[sorted_indices], y=features[sorted_indices], palette="viridis")
        plt.title(f'Feature Importances for Anomaly Detection ({df_name})')
        plt.xlabel('Importance Score')
        plt.ylabel('Features')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{df_name}_feature_importance_anomaly.png'))
        plt.close()
        logging.info(f"Feature importance plot saved for {df_name}.")
    except Exception as e:
        logging.error(f"Could not generate feature importance plot for {df_name}: {e}", exc_info=True)

# --- Main EDA Execution ---
if __name__ == "__main__":
    logging.info("--- Starting Exploratory Data Analysis (EDA) for Pollutant Datasets ---")

    if not os.path.exists(EDA_OUTPUT_DIR):
        os.makedirs(EDA_OUTPUT_DIR)
        logging.info(f"Created EDA output directory: {EDA_OUTPUT_DIR}")

    # Load preprocessed data
    try:
        train_df = pd.read_csv(PREPROCESSED_TRAIN_PATH, parse_dates=['timestamp'])
        logging.info(f"Successfully loaded preprocessed training data from {PREPROCESSED_TRAIN_PATH}. Shape: {train_df.shape}")
    except FileNotFoundError:
        logging.error(f"Training data file not found: {PREPROCESSED_TRAIN_PATH}. Please run the preprocessing script first.")
        train_df = pd.DataFrame() # Ensure it's defined
    except Exception as e:
        logging.error(f"Error loading training data: {e}")
        train_df = pd.DataFrame()

    try:
        test_df = pd.read_csv(PREPROCESSED_TEST_PATH, parse_dates=['timestamp'])
        logging.info(f"Successfully loaded preprocessed testing data from {PREPROCESSED_TEST_PATH}. Shape: {test_df.shape}")
    except FileNotFoundError:
        logging.error(f"Testing data file not found: {PREPROCESSED_TEST_PATH}. Please run the preprocessing script first.")
        test_df = pd.DataFrame() # Ensure it's defined
    except Exception as e:
        logging.error(f"Error loading testing data: {e}")
        test_df = pd.DataFrame()


    # --- Define features for EDA ---
    # Primary pollutant features (already scaled)
    # Engineered features also include these base names, plus _rolling_mean_X and _lag1

    # Collect all numerical features available after preprocessing, excluding time and specific outlier flags
    # This helps build a comprehensive list for correlation and feature importance.
    train_numerical_features = []
    if not train_df.empty:
        train_numerical_features = [col for col in train_df.columns if pd.api.types.is_numeric_dtype(train_df[col]) and col not in ['timestamp', 'is_outlier', 'is_outlier_pm25', 'is_outlier_no2', 'is_outlier_co']] # Add other specific outlier flags if they exist
    
    test_numerical_features = []
    if not test_df.empty:
        test_numerical_features = [col for col in test_df.columns if pd.api.types.is_numeric_dtype(test_df[col]) and col not in ['timestamp']]


    # 1. Time-Series Trends (for primary pollutants)
    if not train_df.empty:
        plot_time_series_trends(train_df, 'Training_Data_Pollutants', PRIMARY_POLLUTANT_FEATURES, EDA_OUTPUT_DIR)
    if not test_df.empty:
        plot_time_series_trends(test_df, 'Testing_Data_Pollutants', PRIMARY_POLLUTANT_FEATURES, EDA_OUTPUT_DIR)

    # 2. Feature Correlations (using all available numerical features)
    if not train_df.empty and train_numerical_features:
        plot_correlation_matrix(train_df, 'Training_Data_Pollutants', train_numerical_features, EDA_OUTPUT_DIR)
    if not test_df.empty and test_numerical_features:
        plot_correlation_matrix(test_df, 'Testing_Data_Pollutants', test_numerical_features, EDA_OUTPUT_DIR)

    # 3. Distribution Analysis (for primary pollutants and some engineered versions)
    dist_features_train = PRIMARY_POLLUTANT_FEATURES[:] # Copy
    if not train_df.empty:
        for f_base in PRIMARY_POLLUTANT_FEATURES:
            if f'{f_base}_rolling_mean_{ROLLING_WINDOW_SIZE}' in train_df.columns:
                dist_features_train.append(f'{f_base}_rolling_mean_{ROLLING_WINDOW_SIZE}')
        plot_distribution_analysis(train_df, 'Training_Data_Pollutants', list(set(dist_features_train)), EDA_OUTPUT_DIR)

    dist_features_test = PRIMARY_POLLUTANT_FEATURES[:]
    if not test_df.empty:
        for f_base in PRIMARY_POLLUTANT_FEATURES:
            if f'{f_base}_rolling_mean_{ROLLING_WINDOW_SIZE}' in test_df.columns:
                dist_features_test.append(f'{f_base}_rolling_mean_{ROLLING_WINDOW_SIZE}')
        plot_distribution_analysis(test_df, 'Testing_Data_Pollutants', list(set(dist_features_test)), EDA_OUTPUT_DIR)


    # 4. Box Plots for Outliers (for primary pollutants)
    if not train_df.empty:
        plot_box_plots(train_df, 'Training_Data_Pollutants', PRIMARY_POLLUTANT_FEATURES, EDA_OUTPUT_DIR)
    if not test_df.empty:
        plot_box_plots(test_df, 'Testing_Data_Pollutants', PRIMARY_POLLUTANT_FEATURES, EDA_OUTPUT_DIR)

    # 5. Missing Data Check (on loaded preprocessed data - should be none if preprocessing worked)
    logging.info("\n--- Missing Values Check (Loaded Preprocessed Data) ---")
    if not train_df.empty: 
        print("Training Data Missing Values Head:\n", train_df.isnull().sum().sort_values(ascending=False).head())
    else: print("Training Data is empty.")
    if not test_df.empty: 
        print("\nTesting Data Missing Values Head:\n", test_df.isnull().sum().sort_values(ascending=False).head())
    else: print("Testing Data is empty.")


    # 6. Feature Importance (using 'is_outlier' from training data)
    if not train_df.empty and 'is_outlier' in train_df.columns and train_numerical_features:
        # Ensure X_train_importance does not contain NaNs, which can happen if some features are all NaN.
        X_train_importance = train_df[train_numerical_features].copy()
        X_train_importance.replace([np.inf, -np.inf], np.nan, inplace=True) # Replace infs
        X_train_importance.dropna(axis=1, how='all', inplace=True) # Drop cols if ALL NaN
        X_train_importance.dropna(axis=0, how='any', inplace=True)  # Drop rows if ANY feature is NaN for RF
        
        if not X_train_importance.empty and X_train_importance.shape[1] > 0:
            # Align y_train_importance with the (potentially now smaller) X_train_importance
            y_train_importance = train_df.loc[X_train_importance.index, 'is_outlier']
            
            if not y_train_importance.empty:
                plot_feature_importance(X_train_importance, y_train_importance, 'Training_Data_Pollutants', EDA_OUTPUT_DIR)
            else:
                logging.warning("y_train_importance became empty after aligning with X_train_importance. Skipping feature importance.")
        else:
            logging.warning("X_train_importance data for feature importance is empty or has no columns after NaN handling. Skipping plot.")
    elif train_df.empty:
        logging.warning("Training data is empty. Skipping feature importance plot.")
    else:
        logging.warning("'is_outlier' column missing, or no numerical features in training data for feature importance plot.")

    # 7. Summary Statistics (for primary pollutants)
    logging.info("\n--- Summary Statistics (Preprocessed Data - Primary Pollutants) ---")
    if not train_df.empty:
        print("\nSummary Statistics for Training Data (Pollutants):")
        print(train_df[PRIMARY_POLLUTANT_FEATURES].describe())
    if not test_df.empty:
        print("\nSummary Statistics for Testing Data (Pollutants):")
        print(test_df[PRIMARY_POLLUTANT_FEATURES].describe())

    logging.info("--- Pollutants EDA Script Finished ---")